In [1]:
# import all the libraries needed

import warnings
warnings.filterwarnings('ignore')

import pickle

import numpy as np
from numpy.linalg import norm

import tensorflow
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.neighbors import NearestNeighbors

import cv2

In [2]:
# read pickle files

features_list = pickle.load(open('images_features_list.pkl', 'rb'))
images_fils = pickle.load(open('images_fils.pkl', 'rb'))

In [3]:
# print shape of features_list

print(np.array(features_list).shape)

(9993, 2048)


In [4]:
# create resnet50 model

model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
model.trainable = False

In [5]:
model = tensorflow.keras.Sequential([model, GlobalMaxPooling2D()])

# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [6]:
# load sample image and extract features

load_image = image.load_img('sample/57064.jpg', target_size = (224, 224))
image_array = image.img_to_array(load_image)
expanded_image_array = np.expand_dims(image_array, axis = 0)
preprocess_image = preprocess_input(expanded_image_array)
result_to_resnet = model.predict(preprocess_image)
result = result_to_resnet.flatten()

1/1 [==============================] - 1s 1s/step


In [7]:
# normalizing

normalized_result = result / norm(result)

In [8]:
# make a features list tree

neighbors = NearestNeighbors(n_neighbors = 5, algorithm = 'brute', metric = 'euclidean')
neighbors.fit(features_list)

distances, indices = neighbors.kneighbors([normalized_result])

print(indices)

[[9898   32 8232 5398  116]]


In [9]:
# shows the images

for file in indices[0][1:6]:
    print(images_fils[file])
    tmp_img = cv2.imread(images_fils[file])
    resize_img = cv2.resize(tmp_img, (512, 512))
    cv2.imshow('Output', resize_img)
    cv2.waitKey(0)

images\1165.jpg
images\33967.jpg
images\27874.jpg
images\12139.jpg
